In [2]:
import json

In [8]:
with open("prompts.json", "r") as file:
    dic = json.loads(file.read())

In [10]:
prompt = dic["prompts"][0]["prompt"]

'You will be provided with a problem statement, a code snippet that supposedly addresses the problem,\nand a catalog of code inconsistencies.\nEvaluation Steps:\n1. Read the problem statement carefully to identify the functionalities required for the\nimplementation.\n2. Read the code snippet and compare it to the problem statement. Check if the code snippet covers\nthe required functionalities.\n3. Output your answer in a JSON format list.\na) If the code snippet is correct, output: ["inconsistency": "None", "severity": "Negligible"].\nb) If the code snippet is incorrect, output the identified inconsistencies and their severity\naccording to the catalog of code inconsistencies. For example: ["inconsistency": "<inconsistency1>",\n"severity": "<severity1>", "inconsistency": "<inconsistency2>", "severity": "<severity2>", ...]\nProblem: {PROBLEM}\nCode Snippet: {CODE}\nTaxonomy of Common Inconsistencies:\n1. Missing dependency declarations: Negligible\n2. No error messages for unexpected 

In [1]:
from CodeJudge import codejudge
from datasets import load_dataset
data = load_dataset("THUDM/humaneval-x", trust_remote_code=True)
data

c:\Users\adity\Desktop\Thesis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'declaration', 'canonical_solution', 'test', 'example_test'],
        num_rows: 164
    })
})

In [2]:
data["test"][0]

{'task_id': 'Python/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'declaration': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(has_close_elements):\n    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    as

In [3]:
problem, code = data["test"][0]["prompt"], data["test"][0]["canonical_solution"]

In [4]:
print(problem)
print(code)

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False



In [5]:
judge = codejudge.CodeJudge(promptLocation="./CodeJudge/prompts.json")

Using CodeJudge Version: CodeJudge 2025.0.alpha


In [6]:
# problem = r'''from typing import List def separate_paren_groups(paren_string: str) -> List[str]: """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to separate those group into separate strings and return the list of those. Separate groups are balanced (each open brace is properly closed) and not nested within each other Ignore any spaces in the input string. >>> separate_paren_groups('( ) (( )) (( )( ))') ['()', '(())', '(()())'] ""'''
# code = r'''result = [] current_string = [] current_depth = 0 for c in paren_string: if c == '(': current_depth += 1 current_string.append(c) elif c == ')': current_depth -= 1 current_string.append(c) if current_depth == 0: result.append(''.join(current_string)) current_string.clear() return result'''

In [7]:
judge.evaluateTaxonomical(problem, code)

{'inconsistencies': [{'inconsistency': 'Logic error: The code does not handle the case where the list has fewer than two elements, which would make it impossible to have close elements.',
   'severity': 'Major'},
  {'inconsistency': 'Edge case not handled: The function does not account for the case where the input list is empty.',
   'severity': 'Small'}],
 'score': 0.44999999999999996}